In [138]:
# open Programming.txt file and read
with open('Knowledge\Dataset.txt', 'r') as file:
    Programming = file.read()


In [102]:
chars = sorted(list(set(Programming)))
vocab_size = len(chars)
print(''.join(chars))
print('Vocab Size:', vocab_size)


 !"'(),-./012347:;?ABCDEFGHIJKLMNOPRSTVWY[]_abcdefghijklmnopqrstuvwxyz—‘’“”…
Vocab Size: 77


#### encoding (tokernizer)

there are different kind of tokenizer methods like google use SentencePiece, OpenAI use tiktoken

In [103]:
# create a mapping from charactors to integers
stoi = {ch:i for i,ch in enumerate(chars)} # make a look up table
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda e: ''.join([itos[c] for c in e])

print(encode('Python'))
print(decode(encode('Python')))

[35, 69, 64, 52, 59, 58]
Python


In [104]:
# encode the entire text  dataset and store it into a torch.tensor
import torch
data = torch.tensor(encode(Programming), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([33703]) torch.int64
tensor([ 1,  1,  1, 27, 49, 56, 56, 59,  7,  1, 67, 49, 56, 47, 59, 57, 49,  9,
         1, 35, 56, 49, 45, 63, 49,  1, 52, 45, 66, 49,  1, 45,  1, 63, 49, 45,
        64,  9,  1, 27, 59, 67,  1, 47, 45, 58,  1, 28,  1, 45, 63, 63, 53, 63,
        64,  1, 69, 59, 65,  1, 64, 59, 48, 45, 69, 19,  0,  0,  1, 38, 52, 45,
        58, 55,  1, 69, 59, 65,  9,  1, 40, 49, 56, 56,  7,  1, 28,  4, 66, 49,
         1, 46, 49, 49, 58,  1, 50, 49, 49, 56, 53, 58, 51,  1, 62, 49, 45, 56,
        56, 69,  1, 48, 59, 67, 58,  1, 56, 45, 64, 49, 56, 69,  7,  1, 45, 58,
        48,  1, 28,  1, 64, 52, 53, 58, 55,  1, 28,  1, 57, 53, 51, 52, 64,  1,
        46, 49,  1, 49, 68, 60, 49, 62, 53, 49, 58, 47, 53, 58, 51,  1, 48, 49,
        60, 62, 49, 63, 63, 53, 59, 58,  9,  1, 28, 64,  4, 63,  1, 46, 49, 49,
        58,  1, 45, 50, 50, 49, 47, 64, 53, 58, 51,  1, 57, 69,  1, 56, 53, 50,
        49,  7,  1, 49, 63, 60, 49, 47, 53, 45, 56, 56, 69,  1, 53, 58,  1, 57,
        

In [105]:
n = int(len(data) * 0.9)
train_data = data[:n]
test_data = data[n:]

In [106]:
block_size = 8
train_data[:block_size+1]

tensor([ 1,  1,  1, 27, 49, 56, 56, 59,  7])

In [107]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")
    

when input is tensor([1]) the target: 1
when input is tensor([1, 1]) the target: 1
when input is tensor([1, 1, 1]) the target: 27
when input is tensor([ 1,  1,  1, 27]) the target: 49
when input is tensor([ 1,  1,  1, 27, 49]) the target: 56
when input is tensor([ 1,  1,  1, 27, 49, 56]) the target: 56
when input is tensor([ 1,  1,  1, 27, 49, 56, 56]) the target: 59
when input is tensor([ 1,  1,  1, 27, 49, 56, 56, 59]) the target: 7


In [108]:
torch.manual_seed(1337)
batch_size = 4 # how many independatn sequence will we process in parallel ?
block_size = 8 # what is the maximum length of a sequence ?

def get_batch(split):
    data = train_data if split == 'train' else val_data
    lx = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in lx])
    y = torch.stack([data[i+1:i+block_size+1] for i in lx])
    return x, y

xb, yb = get_batch('train')
print(xb.shape, yb.shape)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")


torch.Size([4, 8]) torch.Size([4, 8])
when input is [64] the target: 59
when input is [64, 59] the target: 1
when input is [64, 59, 1] the target: 49
when input is [64, 59, 1, 49] the target: 68
when input is [64, 59, 1, 49, 68] the target: 60
when input is [64, 59, 1, 49, 68, 60] the target: 56
when input is [64, 59, 1, 49, 68, 60, 56] the target: 59
when input is [64, 59, 1, 49, 68, 60, 56, 59] the target: 62
when input is [1] the target: 58
when input is [1, 58] the target: 49
when input is [1, 58, 49] the target: 67
when input is [1, 58, 49, 67] the target: 1
when input is [1, 58, 49, 67, 1] the target: 64
when input is [1, 58, 49, 67, 1, 64] the target: 49
when input is [1, 58, 49, 67, 1, 64, 49] the target: 47
when input is [1, 58, 49, 67, 1, 64, 49, 47] the target: 52
when input is [59] the target: 65
when input is [59, 65] the target: 62
when input is [59, 65, 62] the target: 1
when input is [59, 65, 62, 1] the target: 48
when input is [59, 65, 62, 1, 48] the target: 49
when in

In [109]:
print(xb) # our input to the transformer

tensor([[64, 59,  1, 49, 68, 60, 56, 59],
        [ 1, 58, 49, 67,  1, 64, 49, 47],
        [59, 65, 62,  1, 48, 49, 60, 62],
        [45, 58, 64,  1, 53, 57, 60, 45]])


we use bigram language model in neural network

In [110]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 77])
tensor(5.0161, grad_fn=<NllLossBackward0>)

i:zeN 
4J:‘fA.?L7vxA?ai2aj"Ll!d!”CN'lw,lM!JWu'lH)z4z?
r3uhcA;S))P
z7WPP'on1 uGMtfh…AhrInkF?x0iJYGx u


In [111]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [147]:
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


0.6329413652420044


In [148]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


URadz Flozer Distated it there willing enjoy
Withdrawal from aw or stakidacts what they when they're depressed.

What is depression?  What is depression:

What is depression appetite or sleep patterns
What is stress?
A common feeling that affe, present in their reactions to they want to you.ORK
FONALH AND-Z NOT emental Health Weight Aging
Tips and resources to help your child away from your health depression. But if they’ve noticed anything that might be worrying them or haveng depression nor fa


In [121]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


### The mathematical trick in self_attention

In [122]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [123]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))   # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [124]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [86]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [125]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [126]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [127]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [128]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [129]:
k.var()

tensor(1.0449)

In [130]:
q.var()

tensor(1.0700)

In [131]:
wei.var()

tensor(1.0918)

In [132]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [133]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [134]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [135]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [136]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [137]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('Knowledge\Programming.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.212696 M parameters
step 0: train loss 4.6138, val loss 4.6098
step 100: train loss 2.7072, val loss 2.7954
step 200: train loss 2.5307, val loss 2.6434
step 300: train loss 2.4178, val loss 2.5450
step 400: train loss 2.2993, val loss 2.4487
step 500: train loss 2.1599, val loss 2.3489
step 600: train loss 2.0560, val loss 2.2650
step 700: train loss 1.9631, val loss 2.2167
step 800: train loss 1.8512, val loss 2.1324
step 900: train loss 1.7786, val loss 2.1176
step 1000: train loss 1.6930, val loss 2.0446
step 1100: train loss 1.6031, val loss 2.0287
step 1200: train loss 1.5535, val loss 2.0143
step 1300: train loss 1.4914, val loss 1.9937
step 1400: train loss 1.4365, val loss 1.9817
step 1500: train loss 1.3793, val loss 1.9974
step 1600: train loss 1.3363, val loss 1.9978
step 1700: train loss 1.3244, val loss 1.9688
step 1800: train loss 1.2492, val loss 2.0222
step 1900: train loss 1.2361, val loss 2.0006
step 2000: train loss 1.2146, val loss 2.0085
step 2100: train loss 1.